In [62]:
library(metacell)
library(GEOquery)
library(googlesheets4)
library(googledrive)
library(devtools)
library(dplyr)
library(tibble)
library(ggplot2)
library(tidyverse)
library(tgstat)
library(tgconfig)
library(liana)
library(SingleCellExperiment)
library(Seurat)
library(magrittr)

In [63]:
show_resources()

[1] "Default"          "Consensus"        "Baccin2019"       "CellCall"        
 [5] "CellChatDB"       "Cellinker"        "CellPhoneDB"      "CellTalkDB"      
 [9] "connectomeDB2020" "EMBRACE"          "Guide2Pharma"     "HPMR"            
[13] "ICELLNET"         "iTALK"            "Kirouac2010"      "LRdb"            
[17] "Ramilowski2015"   "OmniPath"

In [64]:
show_methods()

[1] "connectome"      "logfc"           "natmi"           "sca"            
 [5] "cellphonedb"     "cytotalk"        "call_squidpy"    "call_cellchat"  
 [9] "call_connectome" "call_sca"        "call_italk"      "call_natmi"

In [65]:
liana_path <- system.file(package = "liana")

In [66]:
op_resource <- select_resource("Consensus")[[1]]

In [67]:
ortholog_resource <- generate_homologs(op_resource = op_resource,
                                       .missing_fun = str_to_title,
                                       max_homologs = 5,
                                       target_organism = 10090) # mouse

One-to-many homolog matches: C4A; C4B; CCL27; CD200R1L; CD55; CEACAM1; CLEC10A; CSF2RB; DSG1; ENO1; GSTP1; HLA-DMB; HLA-E; HLA-F; IFNL3; IL11RA; IL22; KIR3DL3; KLRB1; KNG1; LILRB4; LYZ; MRGPRX2; PILRB; PRND; PRSS3; RNASE2; SAA1; SERPINA1



In [ ]:
ortholog_resource = ortholog_resource %>% 
    rbind(c('S100a8', 'Mcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a9', 'Mcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a8', 'Alcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a9', 'Alcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2'))


In [ ]:
if(!dir.exists("scdb")) dir.create("scdb/")
scdb_init("scdb/", force_reinit=T)
#> initializing scdb to testdb/

In [ ]:
mat = scdb_mat("all_cells_liana")
mc_neutrophil = scdb_mc("neutrophil_mc")
mc_kinetics = scdb_mc("kinetics_mc")
pic_mc = scdb_mc("merged_neutrophil_epithelial_mc")

In [ ]:
doublet_order = c(2, 16)
doublet_clusters = unique(mc_kinetics@annots)[doublet_order]
lymphoid_order = c(25, 26, 27, 21, 20, 28, 24)
myeloid_order = c(31, 32, 29, 34, 36, 33, 35, 22, 19, 23, 30)
epithelial_order = c(11, 13, 12, 14, 18, 15, 17, 10)
stromal_order = c(1, 3, 8, 9, 7)
fibroblast_order = c(5, 4, 6)
order_index = c(lymphoid_order, myeloid_order, epithelial_order, stromal_order, fibroblast_order)
cell_order = unique(mc_kinetics@annots)[order_index]

In [ ]:
cell_order = cell_order[cell_order != "Neutrophils"]

In [10]:
cell_order

[1] "T cells"          "CD8 T cells"      "Treg"             "gd NKT"          
 [5] "NK cells"         "ILC2"             "B cells"          "Monocytes"       
 [9] "MonMacs"          "Perivasc macs"    "Ductal macs"      "cDC1"            
[13] "cDC2"             "pDC"              "Mreg DCs"         "Mast cells"      
[17] "Basophils"        "Progenitor 1"     "Progenitor 2"     "Alveolar"        
[21] "HS"               "Myoepithelial"    "Basal"            "MyoLum"          
[25] "Tumor cells"      "Endothelial"      "Mcam Endothelial" "Smooth muscle"   
[29] "Pericytes"        "Schwann cells"    "fibroblasts"      "Dpp4 fibroblasts"
[33] "CAFs"

In [12]:
sce = scm_export_mat_to_sce("all_cells_liana", add_log_counts=T)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 25.7 GiB”


In [13]:
good_pics = readLines('annotations/neutrophil_good_pics.txt')

In [14]:
sum(good_pics %in% colnames(sce))

[1] 643

In [15]:
length(good_pics)

[1] 643

In [16]:
sce = sce[,colnames(sce) %in% union(union(names(mc_neutrophil@mc), 
                                          names(mc_kinetics@mc[mc_kinetics@annots[mc_kinetics@mc] %in% cell_order])),
                                    good_pics)]

In [17]:
sum(good_pics %in% colnames(sce))

[1] 643

In [429]:
carcinoma_sce = sce[,sce$condition == 'tumor' & sce$Age %in% c('10w', '12w')]

In [430]:
logcounts(carcinoma_sce) = as(logcounts(carcinoma_sce, withDimnames = FALSE), "dgCMatrix")

In [431]:
sum(grepl("__", carcinoma_sce$annots))

[1] 0

In [432]:
mle_res = read.delim("annotations/neutrophil_mle_res.txt", stringsAsFactors = F, row.names=1)

In [433]:
cell_order

[1] "T cells"          "CD8 T cells"      "Treg"             "gd NKT"          
 [5] "NK cells"         "ILC2"             "B cells"          "Monocytes"       
 [9] "MonMacs"          "Perivasc macs"    "Ductal macs"      "cDC1"            
[13] "cDC2"             "pDC"              "Mreg DCs"         "Mast cells"      
[17] "Basophils"        "Progenitor 1"     "Progenitor 2"     "Alveolar"        
[21] "HS"               "Myoepithelial"    "Basal"            "MyoLum"          
[25] "Tumor cells"      "Endothelial"      "Mcam Endothelial" "Smooth muscle"   
[29] "Pericytes"        "Schwann cells"    "fibroblasts"      "Dpp4 fibroblasts"
[33] "CAFs"

In [434]:
cancer_cells = c("Tumor cells")

In [435]:
tans = c("TAN1", "TAN2")

In [436]:
alpha = mle_res[good_pics, "alpha"]; names(alpha) = good_pics
epithelial_mc = mle_res[good_pics, "a_mc"]; names(epithelial_mc) = good_pics
immune_mc = mle_res[good_pics, "b_mc"]; names(immune_mc) = good_pics
merged_names = paste(ifelse(pic_mc@annots[immune_mc] %in% tans, "TANs", pic_mc@annots[immune_mc]), 
                     pic_mc@annots[epithelial_mc], 
                     sep = '__')
names(merged_names) = good_pics

In [437]:
tan_singlets = names(mc_neutrophil@mc[mc_neutrophil@annots[mc_neutrophil@mc] %in% tans])

In [438]:
cell_names = factor(ifelse(colnames(carcinoma_sce) %in% good_pics,
                           merged_names[colnames(carcinoma_sce)],
                           ifelse(colnames(carcinoma_sce) %in% names(mc_neutrophil@mc),
                                  ifelse(colnames(carcinoma_sce) %in% tan_singlets,
                                                  'TANs',
                                                  mc_neutrophil@annots[mc_neutrophil@mc[colnames(carcinoma_sce)]]),
                                  mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]])))

In [439]:
cell_names %>% unique %>% sort

[1] Alveolar                          B cells                          
 [3] Basal                             Basophils                        
 [5] CAFs                              CD8 T cells                      
 [7] cDC1                              cDC2                             
 [9] Cystatin high                     Cystatin high__Progenitor 2      
[11] Cystatin high__Tumor cells        Dpp4 fibroblasts                 
[13] Ductal macs                       Endothelial                      
[15] fibroblasts                       gd NKT                           
[17] HS                                ILC2                             
[19] Mast cells                        Mcam Endothelial                 
[21] MHCII Neut                        MHCII Neut__Alveolar             
[23] MHCII Neut__HS                    MHCII Neut__Myoepithelial        
[25] MHCII Neut__MyoLum                MHCII Neut__Progenitor 1         
[27] MHCII Neut__Progenitor 2          MHCII Neut__Tumor cells          
[29] MonMacs                           Monocytes                        
[31] Mreg DCs                          Myoepithelial                    
[33] MyoLum                            Neut Ptgs2                       
[35] Neut Ptgs2__Alveolar              Neut Ptgs2__Myoepithelial        
[37] Neut Ptgs2__MyoLum                Neut Ptgs2__Progenitor 1         
[39] Neut Ptgs2__Progenitor 2          Neut Ptgs2__Tumor cells          
[41] Neut Socs3                        Neut Socs3__Alveolar             
[43] Neut Socs3__HS                    Neut Socs3__Myoepithelial        
[45] Neut Socs3__MyoLum                Neut Socs3__Progenitor 1         
[47] Neut Socs3__Progenitor 2          Neut Socs3__Tumor cells          
[49] NK cells                          pDC                              
[51] Pericytes                         Perivasc macs                    
[53] Progenitor 1                      Progenitor 2                     
[55] Schwann cells                     Smooth muscle                    
[57] T cells                           TANs                             
[59] TANs__Alveolar                    TANs__HS                         
[61] TANs__Myoepithelial               TANs__MyoLum                     
[63] TANs__Progenitor 1                TANs__Progenitor 2               
[65] TANs__Tumor cells                 Treg                             
[67] Tumor cells                       Young Cystatin high              
[69] Young Cystatin high__Progenitor 1 Young Cystatin high__Tumor cells 
70 Levels: Alveolar B cells Basal Basophils CAFs CD8 T cells cDC1 ... Young Cystatin high__Tumor cells

In [28]:
write_csv(data.frame(cell=colnames(carcinoma_sce), name=cell_names), "annotations/liana_cell_tanmerged_names.csv")

In [440]:
carcinoma_sce@colData$annots = cell_names

In [441]:
sum(good_pics %in% colnames(carcinoma_sce))

[1] 576

In [442]:
sum(grepl("__", carcinoma_sce$annots))

[1] 576

In [32]:
table(carcinoma_sce@colData$annots) %>% sort


                        Basophils       Cystatin high__Progenitor 2 
                                1                                 1 
                   MHCII Neut__HS         MHCII Neut__Myoepithelial 
                                1                                 1 
             Neut Socs3__Alveolar         Neut Socs3__Myoepithelial 
                                1                                 1 
               Neut Socs3__MyoLum                     Schwann cells 
                                1                                 1 
              TANs__Myoepithelial                             Basal 
                                1                                 2 
        Neut Ptgs2__Myoepithelial                Neut Ptgs2__MyoLum 
                                2                                 2 
                   Neut Socs3__HS          Neut Socs3__Progenitor 1 
                                2                                 2 
Young Cystatin high__Progenitor 1

In [ ]:
carcinoma_sce = carcinoma_sce[,table(carcinoma_sce@colData$annots)[carcinoma_sce@colData$annots] >= 30]

In [46]:
tumor_niche = c('CAFs', 'Progenitor 2', 'Tumor cells', 'Pericytes', 'Mcam Endothelial', 'Ductal macs', 'Mreg DCs', 'TANs')

In [47]:
niche_sce = carcinoma_sce[,carcinoma_sce$annots %in% c(tumor_niche, 'TANs__Tumor cells')]

In [48]:
niche_sce$annots = droplevels(niche_sce$annots)

In [49]:
niche_sce$annots %>% unique

[1] Ductal macs       CAFs              Tumor cells       Progenitor 2     
[5] Mcam Endothelial  Pericytes         Mreg DCs          TANs__Tumor cells
[9] TANs             
9 Levels: CAFs Ductal macs Mcam Endothelial Mreg DCs ... Tumor cells

In [58]:
liana_res <- liana_wrap(niche_sce, idents_col = "annots", expr_prop=0.05,
                        resource = 'custom', # resource has to be set to 'custom' to work with external resources
                        external_resource = ortholog_resource , # provide orthologous resource
#                        method=c('sca', 'natmi') # run only with sca and natmi for comp. time
                        )


Running LIANA with `annots` as labels!

Warning message in exec(output, ...):
“12526 genes and/or 0 cells were removed as they had no counts!”
LIANA: LR summary stats calculated!

Now Running: Natmi

Now Running: Connectome

Now Running: Logfc

Now Running: Sca

Now Running: Cellphonedb



In [59]:
liana_res = liana_aggregate(liana_res)

Now aggregating natmi

Now aggregating connectome

Now aggregating logfc

Now aggregating sca

Now aggregating cellphonedb

Aggregating Ranks



In [60]:
write_csv(liana_res, "annotations/liana_results_tanmerged_tumor_pics_niche.csv")